In [1]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Reshape, Conv2D, Concatenate, TimeDistributed, Dense
from tensorflow.keras.losses import MeanAbsoluteError, CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.data import Dataset
from tensorflow import TensorSpec, float32, int32
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

def rand_img(size):
    return np.random.randint(0, 256, size).astype(np.float32) / 255.0

def rand_sentence(len, max):
    return np.random.randint(0, max, len).astype(np.int32)

def one_hot_encoding(sentence, max):
    msg = np.zeros((len(sentence), max), dtype=np.float32)
    for i, v in enumerate(sentence):
        msg[i, v] = 1.0
    return msg

def data_generator(image_size, sentence_len, sentence_max_word, batch_size=32):
    while True:
        x_img = np.zeros((batch_size, image_size[0], image_size[1], image_size[2]), dtype=np.float32)
        x_sen = np.zeros((batch_size, sentence_len), dtype=np.int32)
        y_img = np.zeros((batch_size, image_size[0], image_size[1], image_size[2]), dtype=np.float32)
        y_sen = np.zeros((batch_size, sentence_len, sentence_max_word), dtype=np.float32)
        for i in range(batch_size):
            img = rand_img(image_size)
            sentence = rand_sentence(sentence_len, sentence_max_word)
            sentence_one_hot_encoded = one_hot_encoding(sentence, sentence_max_word)
            x_img[i] = img
            x_sen[i] = sentence
            y_img[i] = img
            y_sen[i] = sentence_one_hot_encoded
        yield (x_img, x_sen), (y_img, y_sen)

def get_model(image_shape, sentence_len, max_word):
    input_img = Input(image_shape)
    input_sen = Input((sentence_len,))
    embed_sen = Embedding(max_word, 100)(input_sen)
    flat_emb_sen = Flatten()(embed_sen)
    flat_emb_sen = Dense(image_shape[0] * image_shape[1] * 3)(flat_emb_sen)
    flat_emb_sen = Reshape((image_shape[0], image_shape[1], 3))(flat_emb_sen)
    trans_input_img = Conv2D(20, 1, activation="relu")(input_img)
    enc_input = Concatenate(axis=-1)([flat_emb_sen, trans_input_img])
    out_img = Conv2D(3, 1, activation='relu', name='image_reconstruction')(enc_input)
    decoder_model = Sequential(name="sentence_reconstruction")
    decoder_model.add(Input(shape=(100, 100, 3)))
    decoder_model.add(Conv2D(1, 1))
    decoder_model.add(Reshape((sentence_len, 100)))
    decoder_model.add(TimeDistributed(Dense(max_word, activation="softmax")))
    out_sen = decoder_model(out_img)
    model = Model(inputs=[input_img, input_sen], outputs=[out_img, out_sen])
    model.compile(optimizer='adam', loss=[MeanAbsoluteError(), CategoricalCrossentropy()],
                  metrics={'sentence_reconstruction': CategoricalAccuracy()})
    encoder_model = Model(inputs=[input_img, input_sen], outputs=[out_img])
    return model, encoder_model, decoder_model

def ascii_encode(message, sentence_len):
    sen = np.zeros((1, sentence_len), dtype=np.int32)
    for i, a in enumerate(message.encode("ascii")):
        sen[0, i] = a
    return sen

def ascii_decode(message):
    return ''.join(chr(int(a)) for a in message[0].argmax(-1))

def main():
    image_shape = (100, 100, 3)
    sentence_len = 100
    max_word = 256
    batch_size = 64
    
    gen = data_generator(image_shape, sentence_len, max_word, batch_size)
    
    output_signature = (
        (TensorSpec(shape=(batch_size, *image_shape), dtype=float32),
         TensorSpec(shape=(batch_size, sentence_len), dtype=int32)),
        (TensorSpec(shape=(batch_size, *image_shape), dtype=float32),
         TensorSpec(shape=(batch_size, sentence_len, max_word), dtype=float32))
    )
    
    dataset = Dataset.from_generator(lambda: gen, output_signature=output_signature)
    
    model, encoder, decoder = get_model(image_shape, sentence_len, max_word)
    try:
        model.load_weights("best_weights.weights.h5")
    except:
        model.fit(dataset, epochs=10, steps_per_epoch=100, callbacks=[
            ModelCheckpoint("best_weights.weights.h5", monitor="loss",
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True)]
        )
    
    img_temp = Image.open("download.png").convert("RGB")  # Ensure image is in RGB format
    res = img_temp.resize((100, 100))
    img = np.expand_dims(img_to_array(res) / 255.0, axis=0)  # Normalized image array
    
    text_to_encode = input("Enter the code you want to encode: ")
    sen = ascii_encode(text_to_encode, sentence_len)
    
    print("The image in which your message will be encoded will be shown now")
    plt.imshow(img[0], interpolation='nearest')
    plt.show()

    y = encoder.predict([img, sen])
    print("The encoded image will be shown now")
    plt.imshow(y[0], interpolation='nearest')
    plt.show()
    
    y_hat = decoder.predict(y)
    print("The decrypted message is: ", ascii_decode(y_hat))

if _name_ == "_main_":
    main()

NameError: name '_name_' is not defined

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Reshape, Conv2D, Concatenate, TimeDistributed, Dense
from tensorflow.keras.losses import MeanAbsoluteError, CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.data import Dataset
from tensorflow import TensorSpec, float32, int32
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

def rand_img(size):
    return np.random.randint(0, 256, size).astype(np.float32) / 255.0

def rand_sentence(len, max):
    return np.random.randint(0, max, len).astype(np.int32)

def one_hot_encoding(sentence, max):
    msg = np.zeros((len(sentence), max), dtype=np.float32)
    for i, v in enumerate(sentence):
        msg[i, v] = 1.0
    return msg

def data_generator(image_size, sentence_len, sentence_max_word, batch_size=32):
    while True:
        x_img = np.zeros((batch_size, image_size[0], image_size[1], image_size[2]), dtype=np.float32)
        x_sen = np.zeros((batch_size, sentence_len), dtype=np.int32)
        y_img = np.zeros((batch_size, image_size[0], image_size[1], image_size[2]), dtype=np.float32)
        y_sen = np.zeros((batch_size, sentence_len, sentence_max_word), dtype=np.float32)
        for i in range(batch_size):
            img = rand_img(image_size)
            sentence = rand_sentence(sentence_len, sentence_max_word)
            sentence_one_hot_encoded = one_hot_encoding(sentence, sentence_max_word)
            x_img[i] = img
            x_sen[i] = sentence
            y_img[i] = img
            y_sen[i] = sentence_one_hot_encoded
        yield (x_img, x_sen), (y_img, y_sen)

def get_model(image_shape, sentence_len, max_word):
    input_img = Input(image_shape)
    input_sen = Input((sentence_len,))
    embed_sen = Embedding(max_word, 100)(input_sen)
    flat_emb_sen = Flatten()(embed_sen)
    flat_emb_sen = Dense(image_shape[0] * image_shape[1] * 3)(flat_emb_sen)
    flat_emb_sen = Reshape((image_shape[0], image_shape[1], 3))(flat_emb_sen)
    trans_input_img = Conv2D(20, 1, activation="relu")(input_img)
    enc_input = Concatenate(axis=-1)([flat_emb_sen, trans_input_img])
    out_img = Conv2D(3, 1, activation='relu', name='image_reconstruction')(enc_input)
    decoder_model = Sequential(name="sentence_reconstruction")
    decoder_model.add(Input(shape=(100, 100, 3)))
    decoder_model.add(Conv2D(1, 1))
    decoder_model.add(Reshape((sentence_len, 100)))
    decoder_model.add(TimeDistributed(Dense(max_word, activation="softmax")))
    out_sen = decoder_model(out_img)
    model = Model(inputs=[input_img, input_sen], outputs=[out_img, out_sen])
    model.compile(optimizer='adam', loss=[MeanAbsoluteError(), CategoricalCrossentropy()],
                  metrics={'sentence_reconstruction': CategoricalAccuracy()})
    encoder_model = Model(inputs=[input_img, input_sen], outputs=[out_img])
    return model, encoder_model, decoder_model

def ascii_encode(message, sentence_len):
    sen = np.zeros((1, sentence_len), dtype=np.int32)
    for i, a in enumerate(message.encode("ascii")):
        sen[0, i] = a
    return sen

def ascii_decode(message):
    return ''.join(chr(int(a)) for a in message[0].argmax(-1))

def main():
    image_shape = (100, 100, 3)
    sentence_len = 100
    max_word = 256
    batch_size = 64
    
    gen = data_generator(image_shape, sentence_len, max_word, batch_size)
    
    output_signature = (
        (TensorSpec(shape=(batch_size, *image_shape), dtype=float32),
         TensorSpec(shape=(batch_size, sentence_len), dtype=int32)),
        (TensorSpec(shape=(batch_size, *image_shape), dtype=float32),
         TensorSpec(shape=(batch_size, sentence_len, max_word), dtype=float32))
    )
    
    dataset = Dataset.from_generator(lambda: gen, output_signature=output_signature)
    
    model, encoder, decoder = get_model(image_shape, sentence_len, max_word)
    try:
        model.load_weights("best_weights.weights.h5")
    except:
        model.fit(dataset, epochs=10, steps_per_epoch=100, callbacks=[
            ModelCheckpoint("best_weights.weights.h5", monitor="loss",
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True)]
        )
    
    img_temp = Image.open("download.png").convert("RGB")  # Ensure image is in RGB format
    res = img_temp.resize((100, 100))
    img = np.expand_dims(img_to_array(res) / 255.0, axis=0)  # Normalized image array
    
    text_to_encode = input("Enter the code you want to encode: ")
    sen = ascii_encode(text_to_encode, sentence_len)
    
    print("The image in which your message will be encoded will be shown now")
    plt.imshow(img[0], interpolation='nearest')
    plt.show()

    y = encoder.predict([img, sen])
    print("The encoded image will be shown now")
    plt.imshow(y[0], interpolation='nearest')
    plt.show()
    
    y_hat = decoder.predict(y)
    print("The decrypted message is: ", ascii_decode(y_hat))

if _name_ == "_main_":
    main()